In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset

import numpy as np
import evaluate

In [3]:
# Load model directly
model_name = "boltuix/bert-lite"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Load dataset
dataset_name = 'ucberkeley-dlab/measuring-hate-speech'
dataset = load_dataset(dataset_name, split='train')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at boltuix/bert-lite and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# Check the shape of the dataset
dataset.shape

(135556, 131)

In [ ]:
# Create a custom dataset from existing dataset. 
# Use 'text' as the input and encode 'hate_speech_score' as the label- if 'hate_speech_score' is > 0.5, it is considered hate speech, so encode it as 1, else 0.
# This classification is taken from the dataset's description:
# hate_speech_score - continuous hate speech measure, where higher = more hateful and lower = less hateful. > 0.5 is approximately hate speech, < -1 is counter or supportive speech, and -1 to +0.5 is neutral or ambiguous.
def encode_labels(example):
    example['label'] = 1 if example['hate_speech_score'] > 0.5 else 0
    return example

encoded_dataset = dataset.map(encode_labels)
encoded_dataset = encoded_dataset.remove_columns(['hate_speech_score'])

In [6]:
# Select only subset of the dataset for training and testing as there are too many examples in the dataset.
subset = encoded_dataset.train_test_split(test_size=0.2)['test']

# Split the dataset into train and test sets
train_test_split = subset.train_test_split(test_size=0.2, seed=42)

train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [7]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'])

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

# Remove the original text column and set the format for PyTorch
tokenized_train_dataset = tokenized_train_dataset.remove_columns(['text'])
tokenized_test_dataset = tokenized_test_dataset.remove_columns(['text'])

tokenized_train_dataset.set_format('torch')
tokenized_test_dataset.set_format('torch')

# Rename the label column to 'labels' for compatibility with Trainer
tokenized_train_dataset = tokenized_train_dataset.rename_column('label', 'labels')
tokenized_test_dataset = tokenized_test_dataset.rename_column('label', 'labels')

Map: 100%|██████████| 5423/5423 [00:01<00:00, 2769.43 examples/s]


In [8]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./fine-tuned-hatebert',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy='epoch',
    save_strategy="epoch",
    load_best_model_at_end=True,
    bf16=True,
    fp16=False,
)

In [9]:
# Define evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)

    acc = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='weighted')
    
    return {
        'accuracy': acc['accuracy'],
        'f1': f1['f1'],
    }

In [10]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

/var/folders/w8/1z9zcf6n61b4xlr7r18jckm40000gn/T/ipykernel_27315/1608242298.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [11]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.329500,0.295891,0.875161,0.873964
2,0.271900,0.278928,0.883644,0.882222
3,0.240000,0.279034,0.886963,0.886733
4,0.214300,0.284807,0.888807,0.888907
5,0.193700,0.291499,0.889176,0.889182


TrainOutput(global_step=6780, training_loss=0.25984863067446906, metrics={'train_runtime': 442.355, 'train_samples_per_second': 245.154, 'train_steps_per_second': 15.327, 'total_flos': 211160058154776.0, 'train_loss': 0.25984863067446906, 'epoch': 5.0})

In [ ]:
# Save the model
save_directory = "./initial-model"
trainer.save_model(save_directory)
tokenizer.save_pretrained(save_directory)

# Directory was moved to ../data/initial-model

('./initial-model/tokenizer_config.json',
 './initial-model/special_tokens_map.json',
 './initial-model/vocab.txt',
 './initial-model/added_tokens.json',
 './initial-model/tokenizer.json')